In [1]:
import csv
from sklearn import preprocessing
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import OneHotEncoder
import scipy.io
import numpy as np
from scipy.stats import mode
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import pandas as pd 

In [2]:
dataSize = 2985217

In [3]:
df = pd.read_csv('train_2016_v2.csv')
print df['logerror'].min()
print df['logerror'].max()

-4.605
4.737


In [4]:
out = tf.net(..)

out = tf.min(out, 4.8)
out = tf.max(out, -4.7)

loss = tf.mse(out)

optimizer.minimize(loss)

# 'propertycountylandusecode',
# 'taxdelinquencyflag',
# 'propertyzoningdesc'

SyntaxError: invalid syntax (<ipython-input-4-76c43d628bf7>, line 1)

In [5]:
properties_df = pd.read_csv('properties_2016.csv')
properties_df.head()

/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


In [26]:
properties_df['regionidzip'].unique()
properties_df['regionidzip'].value_counts()
np.count_nonzero(properties_df['regionidzip'].isnull().values)

13980

In [8]:
properties_df.columns

Index([u'parcelid', u'airconditioningtypeid', u'architecturalstyletypeid',
       u'basementsqft', u'bathroomcnt', u'bedroomcnt', u'buildingclasstypeid',
       u'buildingqualitytypeid', u'calculatedbathnbr', u'decktypeid',
       u'finishedfloor1squarefeet', u'calculatedfinishedsquarefeet',
       u'finishedsquarefeet12', u'finishedsquarefeet13',
       u'finishedsquarefeet15', u'finishedsquarefeet50',
       u'finishedsquarefeet6', u'fips', u'fireplacecnt', u'fullbathcnt',
       u'garagecarcnt', u'garagetotalsqft', u'hashottuborspa',
       u'heatingorsystemtypeid', u'latitude', u'longitude',
       u'lotsizesquarefeet', u'poolcnt', u'poolsizesum', u'pooltypeid10',
       u'pooltypeid2', u'pooltypeid7', u'propertycountylandusecode',
       u'propertylandusetypeid', u'propertyzoningdesc',
       u'rawcensustractandblock', u'regionidcity', u'regionidcounty',
       u'regionidneighborhood', u'regionidzip', u'roomcnt', u'storytypeid',
       u'threequarterbathnbr', u'typeconstructiontyp

## Preprocessing

### Helpers

In [16]:
def dateBucketer(date):
    year, month, day = date.split('-')
    year = int(year)
    month = int(month)
    day = int(day)
    return month

# print dateBucketer("2016-10-14");

def dateList(transactionlist):
    months = []
    for date, error in transactionlist:
        months.append(dateBucketer(date))
    return months

# print(dateList([("2016-10-14",123456), ("2016-11-14",123456)]))

def labelExtractor(month, transactionlist):
    for date, error in transactionlist:
        if (dateBucketer(date) == month):
            return error

### Dictionary: (Key = propertyID, Value=list of transactions)

In [3]:
transactionByProperty = {}
with open('train_2016_v2.csv', 'rb') as csvfile:
    transactionReader = csv.reader(csvfile, delimiter=',')
    length = 0
    firstLine = 0
    
    for row in transactionReader:
        if firstLine == 0:
            firstLine = 1
            continue
        parcelid, logerror, transactiondate = row
        if parcelid in transactionByProperty:
            transactionByProperty[parcelid].append((transactiondate, logerror))
        else:
            transactionByProperty[parcelid] = [(transactiondate, logerror)]
        length +=1
    print "transactions length = " + str(length)

transactions length = 90275


### Build October Data

In [4]:
octoberTransactions = {}
specifiedMonth = 10
for propertyID in transactionByProperty:
    transactionList = transactionByProperty[propertyID]
    if specifiedMonth in dateList(transactionList):
        octoberTransactions[propertyID] = labelExtractor(specifiedMonth, transactionList)

In [5]:
# print(octoberTransactions)
print(len(octoberTransactions.keys()))
# print(octoberTransactions.keys()[:10])

4977


In [6]:
categoryNames = None
with open('properties_2016.csv','r') as csvfile:
    property_reader = csv.DictReader(csvfile)

    for row in property_reader:
        categoryNames = row.keys()
        break

toDeleteCategories = ['propertycountylandusecode',
                      'taxdelinquencyflag',
                      'propertyzoningdesc']
categoricalFeatures = ['airconditioningtypeid',
                       'architecturalstyletypeid',
                       'buildingclasstypeid',
                       'decktypeid',
                       'heatingorsystemtypeid',
                       'propertylandusetypeid',
                       'storytypeid',
                       'typeconstructiontypeid']
numericalFeatures = [key for key in categoryNames if 
                                 key not in categoricalFeatures and key not in toDeleteCategories]

In [7]:
preprocessed_data = []
numeric = numericalFeatures
categorical = categoricalFeatures
numericElemsAvgs = {}
categorElemsModes = {}

with open('properties_2016.csv','r') as csvfile:
    property_reader = csv.DictReader(csvfile)
    count = 0
    for row in property_reader:
        if row['parcelid'] in octoberTransactions:
            preprocessed_data.append(row)

# with open('properties_2016.csv','r') as csvfile:
#     property_reader = csv.DictReader(csvfile)
#     count = 0
#     for row in property_reader:
#         if count > 200000:
#             break;
#         preprocessed_data.append(row)
#         count += 1

for elem in preprocessed_data:
    for i in range(len(numeric)):
        if elem[numeric[i]] != '':
            if numeric[i] == 'hashottuborspa' or numeric[i] == 'fireplaceflag':
                if elem[numeric[i]] == 'TRUE' or elem[numeric[i]] == 'true':
                    elem[numeric[i]] = 1
                else:
                    elem[numeric[i]] = 0
            else:
                elem[numeric[i]] = float(elem.get(numeric[i]))
            
    for i in range(len(categorical)):
        if elem[categorical[i]] != '':
            elem[categorical[i]] = float(elem.get(categorical[i]))

for numer in numeric:
    avg = sum([sample[numer] for sample in preprocessed_data if sample[numer] !=''])/len(preprocessed_data)
    if avg:
        numericElemsAvgs[numer] = avg
    else:
        numericElemsAvgs[numer] = 0
for categ in categorical:
    modeCateg = mode([sample[categ] for sample in preprocessed_data if sample[categ]!=""])[0]
    if modeCateg:
        categorElemsModes[categ] = modeCateg[0] 
    else:
        categorElemsModes[categ] = None

for numer in numeric:
    for sample in preprocessed_data:
        if sample[numer] == '':
            sample[numer] = numericElemsAvgs[numer]
for categ in categorical:
    for sample in preprocessed_data:
        if sample[categ] == '':
            sample[categ] = categorElemsModes[categ]            
# print(numericElemsAvgs)
# print(categorElemsModes)
# print(preprocessed_data[0])

### Convert Dict to Array

In [8]:
octoberDataArray = []
categoriesToKeep = numeric + categorical # + ['OTHER PARAMS LIKE PRICE'] 
for sample in preprocessed_data:
    parcelid = str(int(sample['parcelid']))
    if parcelid in octoberTransactions:
        reformattedSample = [sample[categ] for categ in categoriesToKeep]
        reformattedSample.append(float(octoberTransactions[parcelid]))
        octoberDataArray.append(list(reformattedSample))

In [11]:
def clean(row, numericElemsAvgs, categorElemsModes, numeric, categorical, categoriesToKeep):
    for i in range(len(numeric)):
        if row[numeric[i]] != '':
            if numeric[i] == 'hashottuborspa' or numeric[i] == 'fireplaceflag':
                if row[numeric[i]] == 'TRUE' or row[numeric[i]] == 'true':
                    row[numeric[i]] = 1
                else:
                    row[numeric[i]] = 0
            else:
                row[numeric[i]] = float(row.get(numeric[i]))
    for i in range(len(categorical)):
        if row[categorical[i]] != '':
            row[categorical[i]] = float(row.get(categorical[i]))  

    for numer in numeric:
        if row[numer] == '':
            row[numer] = numericElemsAvgs[numer]
    for categ in categorical:
        if row[categ] == '':
            row[categ] = categorElemsModes[categ]     
    return list([row[categ] for categ in categoriesToKeep])

In [12]:
###TODO: ONE HOT ENCODING :( Takes too much space
# categoryIndices = [categoriesToKeep.index(categ) for categ in categorical]
# enc = OneHotEncoder(categorical_features=categoryIndices)
# enc.fit(octoberDataArray) 
# octoberDataOneHot = enc.transform(octoberDataArray).toarray()
# octoberDataNp = np.array(octoberDataOneHot)

octoberDataNp = np.array(octoberDataArray)
print(octoberDataNp.shape)



(4977, 56)


In [13]:
with open('properties_2016.csv','r') as csvfile:
    property_reader = csv.DictReader(csvfile)
    trainingCollection= []
    for row in property_reader:
        if row['parcelid'] in transactionByProperty:
            cleaned = clean(row, numericElemsAvgs, categorElemsModes, numeric, categorical, categoriesToKeep)
            parcelid = str(int(row['parcelid']))
            cleaned.append(float(transactionByProperty[parcelid][0][1]))
            trainingCollection.append(list(cleaned))

In [50]:
# training = octoberDataNp[:4000]
# validation = octoberDataNp[4000:]
trainingCollectionNp = np.array(trainingCollection)
np.save('train.npy', trainingCollectionNp)
###90150 properties
training = trainingCollectionNp[:80000]
validation = trainingCollectionNp[80000:]

In [15]:
dtrainOctober = xgb.DMatrix(training[:, :-1], label=training[:, -1:])
dvalidationOctober = xgb.DMatrix(validation[:, :-1], label=validation[:, -1:])

In [43]:
param = {'max_depth':2, 'eta':0.03, 'silent':0 }
num_round = 400
bst = xgb.train(param, dtrainOctober, num_round)

In [44]:
preds = bst.predict(dvalidationOctober)
validlabels = dvalidationOctober.get_label()
predList = preds.tolist()
avgError = sum([abs(predList[index] - validlabels[index]) for index in range(len(predList))])/len(predList)
print("avgError = " + str(avgError))

avgError = 0.0664822253609


In [45]:
# #tuning
# bestDepth = 0
# avgErrorTracker = 0.066

# for depth in range(10):
#     maxDepth = depth + 1
#     stepSize = 0.03
#     param = {'max_depth':maxDepth, 'eta': stepSize, 'silent':0 }
#     num_round = 400
#     bst = xgb.train(param, dtrainOctober, num_round)
#     preds = bst.predict(dvalidationOctober)
#     validlabels = dvalidationOctober.get_label()
    
#     predList = preds.tolist()
#     avgError = mean_absolute_error(validlabels, predList)
#     print("depth = " + str(maxDepth))
#     print("avgError = " + str(avgError))
#     print("stepSize = " + str(stepSize))
#     print("num_round = " + str(num_round))
#     if avgError < avgErrorTracker:
#         avgErrorTracker = avgError
#         bestDepth = maxDepth

### Predict Test Set

In [19]:
with open('properties_2016.csv','r') as csvfile:
    property_reader = csv.DictReader(csvfile)
    testData = []
    for row in property_reader:
        testData.append(clean(row, numericElemsAvgs, categorElemsModes, numeric, categorical, categoriesToKeep))

In [20]:
testDataNp = np.array(testData)

In [21]:
testDataNp.shape

(2985217, 55)

In [22]:
# np.save('testData.npy', testDataNp)

In [2]:
# dtest = xgb.DMatrix(testDataNp)
# testPreds = bst.predict(dtest)
testPreds = np.load("NNPredictionsV3/finalPredV3.npy").T

In [23]:
predsCopied = np.vstack((testPreds, testPreds, testPreds, testPreds, testPreds, testPreds)).T

In [4]:
with open('properties_2016.csv','r') as csvfile:
    property_reader = csv.DictReader(csvfile)
    parcelIDs = []
    for row in property_reader:
        parcelIDs.append(int(row['parcelid']))

In [5]:
parcelidNp = np.array(parcelIDs).T
predsCopied.shape

(2985217, 6)

In [6]:
parcelidNp = parcelidNp.reshape(2985217, 1)

In [24]:
combinedPreds = np.hstack((parcelidNp, predsCopied))
print parcelidNp.shape
print predsCopied.shape

(2985217, 1)
(2985217, 6)


In [26]:
np.savetxt("zestimates_results_NNV5.csv", combinedPreds, fmt='%i,%.4f,%.4f,%.4f,%.4f,%.4f,%.4f', delimiter=",", header='ParcelId,201610,201611,201612,201710,201711,201712', comments ="")

In [9]:
testPreds.shape

(1, 2985217)

In [34]:
trainLook = np.load("trainingData.npy")

In [38]:
a = np.array(trainLook[:,-1:]).T
a.sort()
print(a[0][:10])
# sess = tf.Session()
# a = tf.train.Saver([V,W])
# a.restore(sess, "ZestimateWeight.meta")

[-4.605 -4.605 -4.51  -3.194 -2.976 -2.688 -2.397 -2.375 -2.365 -2.365]


In [22]:
testPreds = np.clip(testPreds, -5, 5)

In [21]:
testPreds

array([[ -4.05451655e-03,  -5.37820160e-03,   7.08064735e-02, ...,
          6.55902688e+05,   6.55902688e+05,   6.55902688e+05]], dtype=float32)